In [1]:
%load_ext autoreload

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
%autoreload
from definitions import ROOT_DIR, DATA_DIR
from nyt_api.books_request import get_books, get_list_names

In [4]:
import pandas as pd

In [5]:
list_names = get_list_names()

In [6]:
list_names['results'][0]

{'list_name': 'Combined Print and E-Book Fiction',
 'display_name': 'Combined Print & E-Book Fiction',
 'list_name_encoded': 'combined-print-and-e-book-fiction',
 'oldest_published_date': '2011-02-13',
 'newest_published_date': '2021-05-09',
 'updated': 'WEEKLY'}

In [7]:
list_types = [list_name['list_name_encoded'] for list_name in list_names['results']]

In [8]:
cols = ['title', 'author','publisher', 'description', 'primary_isbn13', 'primary_isbn10']

In [9]:
all_dfs = []
for list_type in list_types[:9]:
    book_json = get_books(list_type)
    book_details = [book['book_details'][0] for book in book_json['results']]
    df_temp = pd.DataFrame(book_details)
    df_temp = df_temp[cols]
    all_dfs.append(df_temp)
    
df = pd.concat(all_dfs)

In [10]:
df = df.drop_duplicates()

In [11]:
df.head()

,title,author,publisher,description,primary_isbn13,primary_isbn10
0,A GAMBLING MAN,David Baldacci,Grand Central,"Aloysius Archer, a World War II veteran, seeks...",9781538719664,None
1,OCEAN PREY,John Sandford,Putnam,The 31st book in the Prey series. When federal...,9780593087022,None
2,THE HILL WE CLIMB,Amanda Gorman,Viking,The poem read on President Joe Biden's Inaugur...,9780593465271,059346527X
3,THE FOUR WINDS,Kristin Hannah,St. Martin's,As dust storms roll during the Great Depressio...,9781250178602,1250178606
4,WHERE THE CRAWDADS SING,Delia Owens,Putnam,In a quiet town on the North Carolina coast in...,9780735219106,0735219109


In [12]:
from sqlalchemy import create_engine

In [13]:
host='192.168.0.109'
user='alex'
password='madison'
db='NYTBooks'

In [14]:
engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db}")

In [15]:
df_books = pd.read_sql_table('book_details_dim', con=engine, columns=cols)

In [16]:
df_books.shape

(0, 6)

In [17]:
df_load = pd.concat([df, df_books]).drop_duplicates(keep=False)

In [18]:
df_load.head()

,title,author,publisher,description,primary_isbn13,primary_isbn10
0,A GAMBLING MAN,David Baldacci,Grand Central,"Aloysius Archer, a World War II veteran, seeks...",9781538719664,None
1,OCEAN PREY,John Sandford,Putnam,The 31st book in the Prey series. When federal...,9780593087022,None
2,THE HILL WE CLIMB,Amanda Gorman,Viking,The poem read on President Joe Biden's Inaugur...,9780593465271,059346527X
3,THE FOUR WINDS,Kristin Hannah,St. Martin's,As dust storms roll during the Great Depressio...,9781250178602,1250178606
4,WHERE THE CRAWDADS SING,Delia Owens,Putnam,In a quiet town on the North Carolina coast in...,9780735219106,0735219109


In [19]:
df_load.shape

(109, 6)

In [20]:
df_load.to_sql('book_details_dim', engine, index=False, if_exists='append')

In [21]:
df_booksr = pd.read_sql_table('book_details_dim', con=engine, columns=cols)

In [22]:
df_booksr.head()

,title,author,publisher,description,primary_isbn13,primary_isbn10
0,A GAMBLING MAN,David Baldacci,Grand Central,"Aloysius Archer, a World War II veteran, seeks...",9781538719664,None
1,OCEAN PREY,John Sandford,Putnam,The 31st book in the Prey series. When federal...,9780593087022,None
2,THE HILL WE CLIMB,Amanda Gorman,Viking,The poem read on President Joe Biden's Inaugur...,9780593465271,059346527X
3,THE FOUR WINDS,Kristin Hannah,St. Martin's,As dust storms roll during the Great Depressio...,9781250178602,1250178606
4,WHERE THE CRAWDADS SING,Delia Owens,Putnam,In a quiet town on the North Carolina coast in...,9780735219106,0735219109


In [23]:
df_booksr.shape

(109, 6)